## Форматирование текста с помощью LLM

### Быстрый старт

1. Убедитесь, что в файле `.env` установлен `OPENAI_API_KEY`
2. Установите зависимости: `uv sync`
3. Выберите ядро "Vika Bot (Python 3.13)" в Jupyter
4. Запустите все ячейки
5. Измените входной текст и экспериментируйте!

### Основные функции

- **Удаление сносок** из текста (например [1][2][3])
- **Улучшение форматирования** Markdown
- **LLM обработка** для финальной полировки текста
- **Подсчет токенов и стоимости** обработки
- **Автоматическая генерация названий** файлов через LLM
- **Организованное сохранение** файлов в папки по датам
- **Валидация имен файлов** и обработка ошибок

### Новое: Умное сохранение файлов

Ноутбук теперь автоматически:
- Создает осмысленные названия файлов на основе содержимого
- Организует файлы в папки по датам: `notebooks/assets/DD.MM.YYYY/`
- Сохраняет результаты в формате `НАЗВАНИЕ_ТЕМЫ.md`


In [22]:
# Импорт библиотек
import os
import re
from openai import OpenAI
from dotenv import load_dotenv
from tokencost import calculate_prompt_cost, calculate_completion_cost
from IPython.display import display, Markdown
import json

load_dotenv()

MODEL = os.getenv("DEFAULT_MODEL", "gpt-4o-mini")
MAX_TOKENS = int(os.getenv("MAX_TOKENS", 2000))
TEMPERATURE = float(os.getenv("TEMPERATURE", 0.3))

print(f"Модель: {MODEL}")
print(f"Максимум токенов: {MAX_TOKENS}")
print(f"Температура: {TEMPERATURE}")


Модель: gpt-4o-mini
Максимум токенов: 2000
Температура: 0.3


In [23]:
def remove_footnotes(text):
    """
    Удаляет сноски вида [1], [2], [1][2][3] из текста
    """
    # Удаляем одиночные и множественные сноски в квадратных скобках
    pattern = r'\[\d+\](?:\[\d+\])*'
    cleaned_text = re.sub(pattern, '', text)
    
    # Убираем лишние пробелы, которые могли остаться
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    
    return cleaned_text.strip()

def improve_markdown_formatting(text):
    """
    Улучшает форматирование markdown
    """
    lines = text.split('\n')
    improved_lines = []
    
    for i, line in enumerate(lines):
        # Убираем лишние пробелы в начале и конце строки
        line = line.strip()
        
        # Добавляем пустые строки перед заголовками если их нет
        if line.startswith('#') and i > 0 and lines[i-1].strip() != '':
            improved_lines.append('')
        
        # Добавляем пустые строки перед блоками кода если их нет
        if line.startswith('```') and i > 0 and lines[i-1].strip() != '':
            improved_lines.append('')
        
        improved_lines.append(line)
        
        # Добавляем пустые строки после заголовков если их нет
        if line.startswith('#') and i < len(lines)-1 and lines[i+1].strip() != '':
            improved_lines.append('')
        
        # Добавляем пустые строки после блоков кода если их нет
        if line.startswith('```') and i < len(lines)-1 and lines[i+1].strip() != '':
            improved_lines.append('')
    
    # Убираем множественные пустые строки
    result_lines = []
    empty_count = 0
    
    for line in improved_lines:
        if line == '':
            empty_count += 1
            if empty_count <= 1:  # Разрешаем максимум одну пустую строку
                result_lines.append(line)
        else:
            empty_count = 0
            result_lines.append(line)
    
    return '\n'.join(result_lines)

# Тест функций
test_text = "Это текст[1][2] с сносками[3].\n\n##Заголовок\nТекст без отступов."
print("Исходный текст:")
print(repr(test_text))
print("\nПосле удаления сносок:")
cleaned = remove_footnotes(test_text)
print(repr(cleaned))
print("\nПосле улучшения форматирования:")
formatted = improve_markdown_formatting(cleaned)
print(repr(formatted))


Исходный текст:
'Это текст[1][2] с сносками[3].\n\n##Заголовок\nТекст без отступов.'

После удаления сносок:
'Это текст с сносками. ##Заголовок Текст без отступов.'

После улучшения форматирования:
'Это текст с сносками. ##Заголовок Текст без отступов.'


In [24]:
# Входной текст для обработки
INPUT_TEXT = """
# Решение проблемы с созданием ключей сервисного аккаунта

Вы столкнулись с политикой организации, которая блокирует создание ключей сервисных аккаунтов из соображений безопасности. Это обычная практика в современных организациях Google Cloud[1][2][3]. Рассмотрим несколько способов решения этой проблемы.

## Способ 1: Использование Application Default Credentials (ADC) для локальной разработки

### Что такое ADC

Application Default Credentials (ADC) — это безопасная альтернатива ключам сервисных аккаунтов для локальной разработки[4][5][6]. ADC автоматически находит учетные данные в зависимости от среды выполнения.

### Настройка ADC

**Шаг 1: Аутентификация через gcloud CLI**

Выполните следующую команду для аутентификации[4][7][8]:

```bash
gcloud auth application-default login
```

Эта команда:
- Откроет браузер для входа в ваш Google аккаунт
- Сохранит учетные данные в `~/.config/gcloud/application_default_credentials.json` (Linux/macOS)[6]
- Предоставит разрешения для работы с Google Cloud APIs

**Шаг 2: Предоставление прав вашему аккаунту**

В Google Cloud Console перейдите в **IAM & Admin** → **IAM** и добавьте вашему Google аккаунту необходимые роли[4]:

1. **Storage Object Admin** - для работы с Cloud Storage
2. При необходимости другие роли для вашего проекта

### Модификация Go кода для работы с ADC

Измените ваш код для использования ADC вместо JSON ключа[6][9]:

```go
package main

import (
    "context"
    "cloud.google.com/go/storage"
    "google.golang.org/api/option"
)

func createStorageClient() (*storage.Client, error) {
    ctx := context.Background()
    
    // Используем Application Default Credentials
    // Убираем option.WithCredentialsFile()
    client, err := storage.NewClient(ctx)
    if err != nil {
        return nil, err
    }
    
    return client, nil
}
```

**Важно:** Убедитесь, что переменная окружения `GOOGLE_APPLICATION_CREDENTIALS` НЕ установлена, чтобы Go SDK использовал ADC[4][6].

## Способ 2: Настройка Workload Identity Federation

Если вам нужна автоматизация (CI/CD), используйте Workload Identity Federation[2][10][11].

### Создание Workload Identity Pool

```bash
gcloud iam workload-identity-pools create "my-pool" \
    --project="your-project-id" \
    --location="global" \
    --display-name="Pool for local development"
```

### Создание провайдера

```bash
gcloud iam workload-identity-pools providers create-oidc "my-provider" \
    --project="your-project-id" \
    --location="global" \
    --workload-identity-pool="my-pool" \
    --display-name="My OIDC provider" \
    --attribute-mapping="google.subject=assertion.sub"
```

## Способ 3: Запрос на изменение организационной политики

Если вам действительно нужны ключи сервисных аккаунтов, обратитесь к администратору организации[1][3][12].

### Роли, необходимые администратору

Администратор должен иметь роль **Organization Policy Administrator** (`roles/orgpolicy.policyAdmin`)[1][3][12].

### Отключение политики для конкретного проекта

Администратор может создать исключение для вашего проекта[3]:

1. Перейти в **IAM & Admin** → **Organization Policies**
2. Найти политику `iam.disableServiceAccountKeyCreation`
3. Нажать **Edit policy**
4. Добавить исключение для вашего проекта

## Способ 4: Использование compute instance с attached service account

Если ваше приложение запущено на GCP Compute Engine или Cloud Run, используйте прикрепленный сервисный аккаунт[13][14][15].

### Настройка на Compute Engine

```go
func createStorageClientOnGCE() (*storage.Client, error) {
    ctx := context.Background()
    
    // На Compute Engine автоматически используется attached service account
    client, err := storage.NewClient(ctx)
    if err != nil {
        return nil, err
    }
    
    return client, nil
}
```

### Получение токена через metadata server

Для отладки можете проверить метаданные инстанса[14][15]:

```bash
curl -H "Metadata-Flavor: Google" \
  "http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/token"
```

## Рекомендуемый подход для вашего случая

**Для локальной разработки:**
1. Используйте `gcloud auth application-default login`[4][7]
2. Предоставьте вашему Google аккаунту роль **Storage Object Admin**
3. Уберите из кода `option.WithCredentialsFile()`

**Для продакшена:**
1. Если на GCP - используйте attached service account
2. Если вне GCP - настройте Workload Identity Federation
3. В крайнем случае - попросите администратора создать исключение для политики

**Преимущества ADC:**
- ✅ Нет необходимости управлять JSON ключами
- ✅ Автоматическое обновление токенов
- ✅ Более безопасно
- ✅ Поддерживается всеми Google Cloud SDK

**Измененный код для работы с ADC:**

```go
package main

import (
    "context"
    "fmt"
    "io"
    "mime/multipart"
    "net/http"
    "time"
    
    "cloud.google.com/go/storage"
    "github.com/gin-gonic/gin"
)

func createStorageClient() (*storage.Client, error) {
    ctx := context.Background()
    
    // ADC автоматически найдет учетные данные
    client, err := storage.NewClient(ctx)
    if err != nil {
        return nil, fmt.Errorf("failed to create storage client: %v", err)
    }
    
    return client, nil
}

func uploadAvatar(client *storage.Client, bucketName string, 
                  objectName string, file multipart.File) (string, error) {
    ctx := context.Background()
    
    wc := client.Bucket(bucketName).Object(objectName).NewWriter(ctx)
    wc.ACL = []storage.ACLRule{{
        Entity: storage.AllUsers, 
        Role:   storage.RoleReader,
    }}
    wc.ContentType = "image/jpeg"
    
    if _, err := io.Copy(wc, file); err != nil {
        return "", err
    }
    
    if err := wc.Close(); err != nil {
        return "", err
    }
    
    return fmt.Sprintf("https://storage.googleapis.com/%s/%s", 
                       bucketName, objectName), nil
}

func main() {
    router := gin.Default()
    router.MaxMultipartMemory = 8 << 20 // 8 MB
    
    router.POST("/users/:userID/avatar", func(c *gin.Context) {
        file, header, err := c.Request.FormFile("avatar")
        if err != nil {
            c.JSON(http.StatusBadRequest, gin.H{"error": "Файл не найден"})
            return
        }
        defer file.Close()
        
        client, err := createStorageClient()
        if err != nil {
            c.JSON(http.StatusInternalServerError, gin.H{"error": "Ошибка подключения к Storage"})
            return
        }
        defer client.Close()
        
        userID := c.Param("userID")
        objectName := fmt.Sprintf("avatars/%s_%d_%s", 
                                 userID, time.Now().Unix(), header.Filename)
        
        avatarURL, err := uploadAvatar(client, "your-bucket-name", objectName, file)
        if err != nil {
            c.JSON(http.StatusInternalServerError, gin.H{"error": "Ошибка загрузки"})
            return
        }
        
        c.JSON(http.StatusOK, gin.H{
            "message": "Аватарка успешно загружена",
            "avatar_url": avatarURL,
        })
    })
    
    router.Run(":8080")
}
```

"""

print(f"Исходный текст ({len(INPUT_TEXT)} символов):")
print(INPUT_TEXT[:500] + "..." if len(INPUT_TEXT) > 500 else INPUT_TEXT)


Исходный текст (6961 символов):

# Решение проблемы с созданием ключей сервисного аккаунта

Вы столкнулись с политикой организации, которая блокирует создание ключей сервисных аккаунтов из соображений безопасности. Это обычная практика в современных организациях Google Cloud[1][2][3]. Рассмотрим несколько способов решения этой проблемы.

## Способ 1: Использование Application Default Credentials (ADC) для локальной разработки

### Что такое ADC

Application Default Credentials (ADC) — это безопасная альтернатива ключам сервисн...


In [25]:
# Шаг 1: Удаление сносок
text_without_footnotes = remove_footnotes(INPUT_TEXT)

In [26]:
# Шаг 2: Улучшение форматирования
text_formatted = improve_markdown_formatting(text_without_footnotes)

In [ ]:
# Функции для создания осмысленного названия файла и структуры папок
import os
from datetime import datetime
from pathlib import Path

def generate_filename_with_llm(text_content, client, model="gpt-4o-mini"):
    """
    Генерирует осмысленное название файла на основе содержимого текста с помощью LLM
    """
    system_prompt = """Ты эксперт по анализу текста. Твоя задача - создать краткое, осмысленное название файла на основе содержимого.

Требования к названию:
1. Максимум 3-4 слова
2. Используй ТОЛЬКО английские буквы, цифры и подчеркивания
3. Название должно отражать основную тему или проблему
4. Используй UPPER_CASE_WITH_UNDERSCORES
5. НЕ используй расширение файла
6. НЕ используй пробелы или специальные символы кроме подчеркивания

Примеры хороших названий:
- GCP_SERVICE_ACCOUNT_SETUP
- DOCKER_DEPLOYMENT_GUIDE  
- API_AUTHENTICATION_FIX
- DATABASE_MIGRATION_STEPS

Верни ТОЛЬКО название файла без дополнительных объяснений."""

    user_prompt = f"""Проанализируй следующий текст и создай для него подходящее название файла:

{text_content[:1000]}{"..." if len(text_content) > 1000 else ""}"""

    try:
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]

        response = client.chat.completions.create(
            model=model,
            messages=messages,
            max_tokens=50,
            temperature=0.1,
        )

        filename = response.choices[0].message.content.strip()
        
        # Очистка названия от потенциально проблематичных символов
        filename = re.sub(r'[^A-Z0-9_]', '', filename.upper())
        
        # Проверка на минимальную длину
        if len(filename) < 3:
            filename = "FORMATTED_TEXT"
            
        return filename
        
    except Exception as e:
        print(f"Ошибка при генерации названия: {e}")
        return "FORMATTED_TEXT"

def create_assets_directory(base_date=None):
    """
    Создает структуру папок assets с текущей датой
    Возвращает путь к созданной папке
    """
    if base_date is None:
        base_date = datetime.now()
    
    # Форматируем дату как DD.MM.YYYY
    date_str = base_date.strftime("%d.%m.%Y")
    
    # Создаем путь
    assets_path = Path("assets") / date_str
    
    # Создаем папки если они не существуют
    assets_path.mkdir(parents=True, exist_ok=True)
    
    return assets_path

def save_formatted_text(content, base_filename=None, client=None, model="gpt-4o-mini"):
    """
    Сохраняет отформатированный текст с автогенерацией названия и структуры папок
    """
    try:
        # Генерируем название файла если не передано
        if base_filename is None and client is not None:
            print("Генерируем название файла...")
            base_filename = generate_filename_with_llm(content, client, model)
            print(f"Сгенерированное название: {base_filename}")
        elif base_filename is None:
            base_filename = "FORMATTED_TEXT"
        
        # Создаем структуру папок
        assets_path = create_assets_directory()
        
        # Полный путь к файлу
        full_filename = f"{base_filename}.md"
        file_path = assets_path / full_filename
        
        # Сохраняем файл
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(content)
        
        print(f"Файл сохранен: {file_path}")
        return str(file_path)
        
    except Exception as e:
        print(f"Ошибка при сохранении файла: {e}")
        return None

# Тест функций
print("Функции для создания файлов готовы к использованию")


Функции для создания файлов готовы к использованию


In [28]:
# Шаг 3: LLM обработка для финальной полировки
client = OpenAI()

SYSTEM_PROMPT = """Ты - эксперт по форматированию технической документации в формате Markdown.

Твоя задача:
1. Улучшить читаемость и структуру текста
2. Исправить грамматические ошибки, если есть
3. Улучшить форматирование Markdown (заголовки, списки, блоки кода)
4. Сохранить всю техническую информацию без изменений
5. Убедиться, что текст логично структурирован

ВАЖНО: Не добавляй новую информацию, только улучшай существующую структуру и читаемость."""

USER_PROMPT = f"""Пожалуйста, улучши форматирование и структуру следующего текста:

{text_formatted}"""

try:
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT}
    ]

    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        max_tokens=MAX_TOKENS,
        temperature=TEMPERATURE,
    )

    final_result = response.choices[0].message.content
    
    print("=== ФИНАЛЬНЫЙ РЕЗУЛЬТАТ ПОСЛЕ LLM ОБРАБОТКИ ===")
    # display(Markdown(final_result))

    # Статистика использования
    if response.usage:
        print(f"\nИспользовано токенов: {response.usage.total_tokens}")
        print(f"Токены промпта: {response.usage.prompt_tokens}")
        print(f"Токены ответа: {response.usage.completion_tokens}")

        try:
            prompt_cost = calculate_prompt_cost(messages, MODEL)
            completion_cost = calculate_completion_cost(final_result, MODEL)
            total_cost = prompt_cost + completion_cost

            print(f"\nСтоимость промпта: ${prompt_cost:.6f}")
            print(f"Стоимость ответа: ${completion_cost:.6f}")
            print(f"Общая стоимость: ${total_cost:.6f}")
        except Exception as cost_error:
            print(f"Не удалось рассчитать стоимость: {cost_error}")

except Exception as e:
    print(f"Ошибка: {e}")
    print("Убедитесь, что OPENAI_API_KEY установлен в .env файле")


=== ФИНАЛЬНЫЙ РЕЗУЛЬТАТ ПОСЛЕ LLM ОБРАБОТКИ ===

Использовано токенов: 3249
Токены промпта: 1627
Токены ответа: 1622

Стоимость промпта: $0.000244
Стоимость ответа: $0.000973
Общая стоимость: $0.001217


In [ ]:
# Тестирование функций создания файлов (опционально)
test_functions = True  # Измените на False чтобы отключить

if test_functions:
    print("🧪 ТЕСТИРОВАНИЕ НОВЫХ ФУНКЦИЙ")
    print("="*40)
    
    # Тест определения корректного пути
    print("\n0. Проверка путей:")
    current_dir = Path().resolve()
    print(f"   Текущая директория: {current_dir}")
    
    # Тест создания папки
    print("\n1. Тест создания структуры папок:")
    test_assets_path = create_assets_directory()
    print(f"   Создана папка: {test_assets_path}")
    print(f"   Папка существует: {test_assets_path.exists()}")
    
    # Тест генерации названия файла
    if 'client' in locals():
        print("\n2. Тест генерации названия файла:")
        test_text = "Руководство по настройке Docker контейнеров для Python приложений с использованием uv и Poetry"
        test_filename = generate_filename_with_llm(test_text, client, MODEL)
        print(f"   Исходный текст: {test_text}")
        print(f"   Сгенерированное название: {test_filename}")
    else:
        print("\n2. Пропускаем тест генерации (нет OpenAI клиента)")
    
    # Тест полного сохранения
    print("\n3. Тест полного сохранения:")
    test_content = "# Тестовый документ по Docker\n\nЭто руководство по настройке Docker контейнеров для Python приложений.\n\n## Основные шаги\n\n1. Создание Dockerfile\n2. Настройка docker-compose\n3. Деплой в продакшн"
    
    if 'client' in locals():
        test_saved_path = save_formatted_text(test_content, None, client, MODEL)
    else:
        test_saved_path = save_formatted_text(test_content, "TEST_DOCKER_GUIDE", None)
    
    if test_saved_path:
        print(f"   ✅ Тестовый файл сохранен: {test_saved_path}")
    else:
        print("   ❌ Ошибка при сохранении тестового файла")
        
    print("\n✨ Тестирование завершено!")
else:
    print("💡 Для тестирования функций измените test_functions = True")


In [29]:
# Сохранение результата с автоматической генерацией названия и структуры папок
if 'final_result' in locals():
    print("Результат сохранен в переменной 'final_result'")
    print(f"Длина финального текста: {len(final_result)} символов")
    
    # Настройки сохранения
    auto_save = True  # Автоматическое сохранение
    generate_filename = True  # Генерировать название через LLM
    manual_filename = None  # Можно указать вручную, например "MY_CUSTOM_NAME"
    
    if auto_save:
        print("\n" + "="*50)
        print("АВТОМАТИЧЕСКОЕ СОХРАНЕНИЕ ФАЙЛА")
        print("="*50)
        
        try:
            if generate_filename and 'client' in locals():
                # Используем LLM для генерации названия
                saved_path = save_formatted_text(
                    content=final_result,
                    base_filename=manual_filename,  # None = автогенерация
                    client=client,
                    model=MODEL
                )
            else:
                # Используем стандартное название или ручное
                filename = manual_filename if manual_filename else "FORMATTED_TEXT"
                saved_path = save_formatted_text(
                    content=final_result,
                    base_filename=filename,
                    client=None
                )
            
            if saved_path:
                print(f"\n✅ Файл успешно сохранен!")
                print(f"📁 Путь: {saved_path}")
                
                # Дополнительная информация
                file_size = len(final_result.encode('utf-8'))
                print(f"📊 Размер файла: {file_size:,} байт")
                print(f"📝 Строк текста: {final_result.count(chr(10)) + 1}")
            else:
                print("❌ Ошибка при сохранении файла")
                
        except Exception as e:
            print(f"❌ Критическая ошибка при сохранении: {e}")
            print("Попробуйте сохранить файл вручную")
    else:
        print("\n💡 Автосохранение отключено. Для включения измените auto_save = True")
        
else:
    print("❌ Финальный результат не был создан. Проверьте выполнение предыдущих ячеек.")


Результат сохранен в переменной 'final_result'
Длина финального текста: 6745 символов

АВТОМАТИЧЕСКОЕ СОХРАНЕНИЕ ФАЙЛА
Генерируем название файла...
Сгенерированное название: SERVICE_ACCOUNT_KEY_ISSUE_RESOLUTION
Файл сохранен: notebooks/assets/24.09.2025/SERVICE_ACCOUNT_KEY_ISSUE_RESOLUTION.md

✅ Файл успешно сохранен!
📁 Путь: notebooks/assets/24.09.2025/SERVICE_ACCOUNT_KEY_ISSUE_RESOLUTION.md
📊 Размер файла: 8,786 байт
📝 Строк текста: 208
